In [1]:
import numpy as np
import re

from pol_analysis.polData import PolData
from pol_analysis.runPhot import RunPhot
from pol_analysis.getStokes import GetStokes

In [2]:
#Position of the star we will use to measure the PSF. Ideally, we'll upgrade this to use more than one star, but one is ok for now. 
star_ex_ref = 1145
star_ey_ref = 37.5

In [3]:
#Position of the BHD. 
bhd_ex_ref = 1021.5
bhd_ey_ref = 70.0

In [4]:
#Create the data object. This will run the mask creation and the cosmic ray rejection.  
bhd = PolData("W0019-1046","R_SPECIAL")#, force_new=True)

In [5]:
#Create the photometry object and run the common steps. 
phot = RunPhot(bhd)#, force_new=True)
phot.subtract_background(box_size=(25,25))
phot.get_source_positions(box_size=11)

In [6]:
ob_combs = list()
for ob_pair in bhd.ob_pairs:
    ob_combs.append([[ob_pair[0]],[ob_pair[1]]])
ob_combs.append([[None], [None]])

ob_combs

[[[3565057], [60201]], [[3565577], [60202]], [[None], [None]]]

### First OB MJD pair. 

In [7]:
#Set the pair values.
ob_id, mjd = bhd.ob_pairs[0]
ob_ids = [ob_id]
mjds = [mjd]

#Get the seeing. 
fnames = bhd.list_of_filenames(ob_ids=ob_ids, mjds=mjds)
stddev_0 = np.ones(len(fnames)) * 1.4 / 2.355
phot.find_seeing(star_ex_ref, star_ey_ref, stddev_0=stddev_0, show_plots=False, ob_ids=ob_ids, mjds=mjds)
print(phot.seeing)

#Get the photometry convolving to a common PSF. 
phot.get_phot(r_ap=1.0, force_new=True, apply_convolution=True, ob_ids=ob_ids, mjds=mjds)

#Measure the stokes and polarization parameters. 
stk_obj = GetStokes(bhd)
stk_obj.get_pol(ob_ids=ob_ids, mjds=mjds)

#Get the polarization values for the BHD. 
epos_ref = np.loadtxt("{}/{}".format(bhd.phot_folder, re.sub(".fits", ".epos", fnames[0])))
dist2 = (epos_ref[:,0]-bhd_ex_ref)**2 + (epos_ref[:,1]-bhd_ey_ref)**2
k = np.argmin(dist2)
print("{:5.2f} {:5.2f} {:6.1f} {:6.1f}".format(stk_obj.pol_frac[k]*100, stk_obj.epol_frac[k]*100, stk_obj.pol_angle[k], stk_obj.epol_angle[k]))

[1.11919084 1.0490138  1.06496564 1.25594511 1.21634144 1.15884546
 1.05958439 1.49308463]
 7.09  1.03  166.7    4.4


### Second Pair

In [8]:
#Set the pair values.
ob_id, mjd = bhd.ob_pairs[1]
ob_ids = [ob_id]
mjds = [mjd]

#Get the seeing. 
fnames = bhd.list_of_filenames(ob_ids=ob_ids, mjds=mjds)
stddev_0 = np.ones(len(fnames)) * 3.0 #1.4 / 2.355
phot.find_seeing(star_ex_ref, star_ey_ref, stddev_0=stddev_0, show_plots=False, ob_ids=ob_ids, mjds=mjds)
print(phot.seeing)

#Get the photometry convolving to a common PSF. 
phot.get_phot(r_ap=1.0, force_new=True, apply_convolution=True, ob_ids=ob_ids, mjds=mjds)

#Measure the stokes and polarization parameters. 
stk_obj = GetStokes(bhd)
stk_obj.get_pol(ob_ids=ob_ids, mjds=mjds)

#Get the polarization values for the BHD. 
epos_ref = np.loadtxt("{}/{}".format(bhd.phot_folder, re.sub(".fits", ".epos", fnames[0])))
dist2 = (epos_ref[:,0]-bhd_ex_ref)**2 + (epos_ref[:,1]-bhd_ey_ref)**2
k = np.argmin(dist2)
print("{:5.2f} {:5.2f} {:6.1f} {:6.1f}".format(stk_obj.pol_frac[k]*100, stk_obj.epol_frac[k]*100, stk_obj.pol_angle[k], stk_obj.epol_angle[k]))

[0.92754894 0.94732953 0.70921153 0.77163514 0.82352423 0.7942171
 0.79172375 0.823943  ]
 5.31  0.81  162.0    4.9


### Combined

In [9]:
#Set the pair values.
ob_ids = None
mjds = None

#Get the seeing. 
fnames = bhd.list_of_filenames(ob_ids=ob_ids, mjds=mjds)
stddev_0 = np.ones(len(fnames)) * 1.4 / 2.355
stddev_0[8:] = 3.0
phot.find_seeing(star_ex_ref, star_ey_ref, stddev_0=stddev_0, show_plots=False, ob_ids=ob_ids, mjds=mjds)
print(phot.seeing)

#Get the photometry convolving to a common PSF. 
phot.get_phot(r_ap=1.0, force_new=True, apply_convolution=True, ob_ids=ob_ids, mjds=mjds)

#Measure the stokes and polarization parameters. 
stk_obj = GetStokes(bhd)
stk_obj.get_pol(ob_ids=ob_ids, mjds=mjds)

#Get the polarization values for the BHD. 
epos_ref = np.loadtxt("{}/{}".format(bhd.phot_folder, re.sub(".fits", ".epos", fnames[0])))
dist2 = (epos_ref[:,0]-bhd_ex_ref)**2 + (epos_ref[:,1]-bhd_ey_ref)**2
k = np.argmin(dist2)
print("{:5.2f} {:5.2f} {:6.1f} {:6.1f}".format(stk_obj.pol_frac[k]*100, stk_obj.epol_frac[k]*100, stk_obj.pol_angle[k], stk_obj.epol_angle[k]))

[1.11919084 1.0490138  1.06496564 1.25594511 1.21634144 1.15884546
 1.05958439 1.49308463 0.92754894 0.94732953 0.70921153 0.77163514
 0.82352423 0.7942171  0.79172375 0.823943  ]
 6.29  0.77  164.6    3.4
